In [1]:
import pandas as pd
in_file = "./soccer_2019_2014.csv"
u18_df = pd.read_csv(in_file,encoding="UTF-8")

In [2]:
#列の順番を整理する time,playerの列を削除
u18_df=u18_df.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','url','results_away','results_home','goal_away','goal_home']]
u18_df.head(2)
# u18_df.shape

,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home
0,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資
1,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None


In [3]:
# スコアがNone(=未実施)の試合の行を削除
u18_df=u18_df[u18_df['results_away'] != "None"]
u18_df.shape

(1047, 13)

In [4]:
# 年が変わるときの行(カラム名が入っている)を削除(一応...)
u18_df=u18_df[u18_df['results_away'] != "results_away"]
u18_df.shape

(1047, 13)

In [5]:
# # yearに"2002"が入ってしまっているものの対処(未実装)
# u18_df=u18_df[u18_df['year'].str.match('.*,2002')!= "True"]
# u18_df.shape

In [6]:
#idの混同を避けるため、idとyearを合わせる
u18_df["merg_id"] = u18_df["year"].astype(str)+u18_df["id"].astype(str)
u18_df["merg_id"] = u18_df["merg_id"].astype(int)
u18_df.dtypes

id               int64
leagu_name      object
year             int64
month           object
day             object
round           object
team_home       object
team_away       object
url             object
results_away    object
results_home    object
goal_away       object
goal_home       object
merg_id          int32
dtype: object

In [7]:
# ここからaway

In [8]:
#awayチームの得点者とその試合のidのみを抜き出す
df_spr_away = pd.DataFrame(u18_df['goal_away'].str.split(',', expand=True))
df_spr_away["merg_id"] =u18_df["merg_id"]

In [9]:
# 先ほどの表から、1ゴール目のデータだけの表を作り、ゴールのない試合は削除
df_spr_away0 = pd.DataFrame({
    "goal":df_spr_away[0],
    "merg_id":df_spr_away["merg_id"]
})
df_spr_away0=df_spr_away0[df_spr_away0['goal'] != "None"] # df_spr_away0.dropna(inplace=True) はもともと"None"なので、使えない
df_spr_away0["count_away"]=int(1)
df_spr_away0.head(3)

,goal,merg_id,count_away
0,26分 渡邊 翔耶,2019101,1
1,25分 染野 唯月,2019102,1
2,17分 青島 太一,2019103,1


In [10]:
#2ゴール目以降のデータも追加
for repeat in range(1, ( (len(df_spr_away.columns)) -1) ):
    df_spr_away_n = pd.DataFrame({
    "goal":df_spr_away[repeat],
    "merg_id":df_spr_away["merg_id"],
    })
    df_spr_away_n.dropna(inplace=True)
    df_spr_away_n["count_away"]=int(1+repeat)
    df_spr_away0 = pd.concat([df_spr_away0,df_spr_away_n],sort=True,copy=False,ignore_index=True)

df_spr_away0

,count_away,goal,merg_id
0,1,26分 渡邊 翔耶,2019101
1,1,25分 染野 唯月,2019102
2,1,17分 青島 太一,2019103
3,1,32分 後藤 健太,2019104
4,1,55分 福井 啓太,2019202
5,1,14分 佐藤 優斗,2019203
6,1,64分 細谷 真大,2019204
7,1,6分 田中 翔太,2019301
8,1,18分 堀井 真海,2019302
9,1,8分 ノリエガ・エリック,2019303


In [11]:
#選手名と時間を分ける
df_spr_away0 = pd.concat([df_spr_away0,df_spr_away0["goal"].str.split("分",expand=True)],axis=1).drop("goal", axis=1)
df_spr_away0.rename(columns={0: 'time', 1: 'player'}, inplace=True)

# additional timeで別カラム
df_spr_away0 = pd.concat([df_spr_away0,df_spr_away0["time"].str.split("+",expand=True)],axis=1).drop("time", axis=1).fillna(-1)
df_spr_away0.rename(columns={0: 'n_time', 1: 'a_time'}, inplace=True)
df_spr_away0["n_time"] =df_spr_away0["n_time"].astype(int)
df_spr_away0["n_time"] =df_spr_away0["n_time"].astype(int)

# df_spr_away0["a_time"] =pd.to_numeric(df_spr_away0["a_time"], errors='coerce')

df_spr_away0

,count_away,merg_id,player,2,n_time,a_time
0,1,2019101,渡邊 翔耶,-1,26,-1
1,1,2019102,染野 唯月,-1,25,-1
2,1,2019103,青島 太一,-1,17,-1
3,1,2019104,後藤 健太,-1,32,-1
4,1,2019202,福井 啓太,-1,55,-1
5,1,2019203,佐藤 優斗,-1,14,-1
6,1,2019204,細谷 真大,-1,64,-1
7,1,2019301,田中 翔太,-1,6,-1
8,1,2019302,堀井 真海,-1,18,-1
9,1,2019303,ノリエガ・エリック,-1,8,-1


In [12]:
# 名前に「分」が入ってる人
df_spr_away0.loc[ (df_spr_away0[2] != -1 ), "player"] = df_spr_away0["player"].astype(str) + str("分") + df_spr_away0[2].astype(str)
df_spr_away0=df_spr_away0.drop(2, axis=1)
df_spr_away0

,count_away,merg_id,player,n_time,a_time
0,1,2019101,渡邊 翔耶,26,-1
1,1,2019102,染野 唯月,25,-1
2,1,2019103,青島 太一,17,-1
3,1,2019104,後藤 健太,32,-1
4,1,2019202,福井 啓太,55,-1
5,1,2019203,佐藤 優斗,14,-1
6,1,2019204,細谷 真大,64,-1
7,1,2019301,田中 翔太,6,-1
8,1,2019302,堀井 真海,18,-1
9,1,2019303,ノリエガ・エリック,8,-1


In [13]:
#ここからhome

In [14]:
#homeチームの得点者とその試合のidのみを抜き出す
df_spr_home = pd.DataFrame(u18_df['goal_home'].str.split(',', expand=True))
df_spr_home["merg_id"] =u18_df["merg_id"]

In [15]:
# 先ほどの表から、1ゴール目のデータだけの表を作り、ゴールのない試合は削除
df_spr_home0 = pd.DataFrame({
    "goal":df_spr_home[0],
    "merg_id":df_spr_home["merg_id"]
})
df_spr_home0=df_spr_home0[df_spr_home0['goal'] != "None"] # df_spr_home0.dropna(inplace=True) はもともと"None"なので、使えない
df_spr_home0["count_home"]=int(1)

#2ゴール目以降のデータも追加
for repeat in range(1, ( (len(df_spr_home.columns)) -1) ):
    df_spr_home_n = pd.DataFrame({
    "goal":df_spr_home[repeat],
    "merg_id":df_spr_home["merg_id"],
    })
    df_spr_home_n.dropna(inplace=True)
    df_spr_home_n["count_home"]=int(1+repeat)
    df_spr_home0 = pd.concat([df_spr_home0,df_spr_home_n],sort=True,copy=False,ignore_index=True)

df_spr_home0

,count_home,goal,merg_id
0,1,52分 石田 侑資,2019101
1,1,45+2分 大澤 朋也,2019105
2,1,44分 森山 一斗,2019201
3,1,45+2分 植松 建斗,2019202
4,1,16分 箱﨑 拓,2019205
5,1,47分 髙田 颯也,2019301
6,1,52分 松本 岳士,2019303
7,1,58分 藤原 健介,2019304
8,1,9分 奥田 陽琉,2019305
9,1,27分 揖斐 俊斗,2019401


In [16]:
#選手名と時間を分ける
# df3 = pd.concat([df['local'], df['domain'].str.split('.', expand=True)], axis=1)
df_spr_home0 = pd.concat([df_spr_home0,df_spr_home0["goal"].str.split("分",expand=True)],axis=1).drop("goal", axis=1)
df_spr_home0.rename(columns={0: 'time', 1: 'player'}, inplace=True)

# additional timeで別カラム
df_spr_home0 = pd.concat([df_spr_home0,df_spr_home0["time"].str.split("+",expand=True)],axis=1).drop("time", axis=1).fillna(-1)
df_spr_home0.rename(columns={0: 'n_time', 1: 'a_time'}, inplace=True)
df_spr_home0["n_time"] =df_spr_home0["n_time"].astype(int)
df_spr_home0["a_time"] =df_spr_home0["a_time"].astype(int)

df_spr_home0

,count_home,merg_id,player,2,n_time,a_time
0,1,2019101,石田 侑資,-1,52,-1
1,1,2019105,大澤 朋也,-1,45,2
2,1,2019201,森山 一斗,-1,44,-1
3,1,2019202,植松 建斗,-1,45,2
4,1,2019205,箱﨑 拓,-1,16,-1
5,1,2019301,髙田 颯也,-1,47,-1
6,1,2019303,松本 岳士,-1,52,-1
7,1,2019304,藤原 健介,-1,58,-1
8,1,2019305,奥田 陽琉,-1,9,-1
9,1,2019401,揖斐 俊斗,-1,27,-1


In [17]:
# 名前に「分」が入ってる人
df_spr_home0.loc[ (df_spr_home0[2] != -1 ), "player"] = df_spr_home0["player"].astype(str) + str("分") + df_spr_home0[2].astype(str)
df_spr_home0=df_spr_home0.drop(2, axis=1)
df_spr_home0

,count_home,merg_id,player,n_time,a_time
0,1,2019101,石田 侑資,52,-1
1,1,2019105,大澤 朋也,45,2
2,1,2019201,森山 一斗,44,-1
3,1,2019202,植松 建斗,45,2
4,1,2019205,箱﨑 拓,16,-1
5,1,2019301,髙田 颯也,47,-1
6,1,2019303,松本 岳士,52,-1
7,1,2019304,藤原 健介,58,-1
8,1,2019305,奥田 陽琉,9,-1
9,1,2019401,揖斐 俊斗,27,-1


In [18]:
#ここからまとめて

In [19]:
#homeとawayを合体 #並び変える（ソート）
df_merge = pd.concat([df_spr_away0,df_spr_home0],sort=True,copy=False,ignore_index=True)
# .sort_values(["id","n_time","a_time"]).reset_index(drop=True)
df_merged = pd.merge(df_merge,u18_df, on="merg_id").sort_values(["year","id","n_time","a_time"],ascending=[False,True,True,True]).reset_index(drop=True)

df_merged["differ"] =df_merged["id"].diff().fillna(1)
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,NaN,2019101,26,渡邊 翔耶,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,1.0
1,-1,NaN,1.0,2019101,52,石田 侑資,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,0.0
2,4,2.0,NaN,2019101,90,野中 悠翔,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,0.0
3,-1,1.0,NaN,2019102,25,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,1.0
4,-1,2.0,NaN,2019102,79,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,0.0
5,-1,3.0,NaN,2019102,85,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,0.0
6,-1,1.0,NaN,2019103,17,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"17分 青島 太一,81分 青島 太一",None,1.0
7,-1,2.0,NaN,2019103,81,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"17分 青島 太一,81分 青島 太一",None,0.0
8,-1,1.0,NaN,2019104,32,後藤 健太,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,1.0
9,-1,2.0,NaN,2019104,86,古澤 ナベル慈宇,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,0.0


In [20]:
#idが変わる試合 かつ counthomeがnullのもの(=相手が先制しているので、counthomeは0になるべき)
df_merged.loc[(df_merged["differ"] != 0.0) &  (df_merged["count_home"].isnull() ), "count_home"] = 0
#awayも同様
df_merged.loc[(df_merged["differ"] != 0.0) &  (df_merged["count_away"].isnull() ), "count_away"] = 0

df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,0.0,2019101,26,渡邊 翔耶,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,1.0
1,-1,NaN,1.0,2019101,52,石田 侑資,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,0.0
2,4,2.0,NaN,2019101,90,野中 悠翔,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,0.0
3,-1,1.0,0.0,2019102,25,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,1.0
4,-1,2.0,NaN,2019102,79,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,0.0
5,-1,3.0,NaN,2019102,85,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,0.0
6,-1,1.0,0.0,2019103,17,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"17分 青島 太一,81分 青島 太一",None,1.0
7,-1,2.0,NaN,2019103,81,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"17分 青島 太一,81分 青島 太一",None,0.0
8,-1,1.0,0.0,2019104,32,後藤 健太,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,1.0
9,-1,2.0,NaN,2019104,86,古澤 ナベル慈宇,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,0.0


In [21]:
#残りのNaNは、相手ゴール分なので、一つ上の行のデータで埋める
df_merged["count_away"]=df_merged["count_away"].fillna(method='ffill')
df_merged["count_home"]=df_merged["count_home"].fillna(method='ffill')
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,0.0,2019101,26,渡邊 翔耶,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,1.0
1,-1,1.0,1.0,2019101,52,石田 侑資,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,0.0
2,4,2.0,1.0,2019101,90,野中 悠翔,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,0.0
3,-1,1.0,0.0,2019102,25,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,1.0
4,-1,2.0,0.0,2019102,79,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,0.0
5,-1,3.0,0.0,2019102,85,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,0.0
6,-1,1.0,0.0,2019103,17,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"17分 青島 太一,81分 青島 太一",None,1.0
7,-1,2.0,0.0,2019103,81,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"17分 青島 太一,81分 青島 太一",None,0.0
8,-1,1.0,0.0,2019104,32,後藤 健太,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,1.0
9,-1,2.0,0.0,2019104,86,古澤 ナベル慈宇,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,0.0


In [22]:
#試合情報を結合
# df_merged = pd.merge(df_merge,u18_df, on="id").sort_values("year",ascending=False)
df_merged["goal_away"]=df_merged["count_away"].astype(int)
df_merged["goal_home"]=df_merged["count_home"].astype(int)
# del df_merged["count_away"]
# del df_merged["count_home"]
df_merged

,a_time,count_away,count_home,merg_id,n_time,player,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,differ
0,-1,1.0,0.0,2019101,26,渡邊 翔耶,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,1,0,1.0
1,-1,1.0,1.0,2019101,52,石田 侑資,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,1,1,0.0
2,4,2.0,1.0,2019101,90,野中 悠翔,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,2,1,0.0
3,-1,1.0,0.0,2019102,25,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,1,0,1.0
4,-1,2.0,0.0,2019102,79,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,2,0,0.0
5,-1,3.0,0.0,2019102,85,染野 唯月,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,3,0,0.0
6,-1,1.0,0.0,2019103,17,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,1,0,1.0
7,-1,2.0,0.0,2019103,81,青島 太一,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,2,0,0.0
8,-1,1.0,0.0,2019104,32,後藤 健太,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,1,0,1.0
9,-1,2.0,0.0,2019104,86,古澤 ナベル慈宇,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,2,0,0.0


In [23]:
#additional timeをプラス表記に
df_merged["time"] = df_merged["n_time"]
df_merged.loc[ (df_merged["a_time"] !=-1 ), "time"] = df_merged["n_time"].astype(str) + str("+") + df_merged["a_time"].astype(str)

df_merged=df_merged.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','url','results_away','results_home','goal_away','goal_home','time','player']]
df_merged

,id,leagu_name,year,month,day,round,team_home,team_away,url,results_away,results_home,goal_away,goal_home,time,player
0,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,1,0,26,渡邊 翔耶
1,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,1,1,52,石田 侑資
2,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,1,2,1,90+4,野中 悠翔
3,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,1,0,25,染野 唯月
4,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,2,0,79,染野 唯月
5,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,3,0,3,0,85,染野 唯月
6,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,1,0,17,青島 太一
7,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,2,0,81,青島 太一
8,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,1,0,32,後藤 健太
9,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,http://www.jfa.jp/match/takamado_jfa_u18_premi...,2,0,2,0,86,古澤 ナベル慈宇


In [24]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'python', 'dataformat.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'python', 'dataformat.ipynb'], returncode=0)

In [25]:
# 出力CSVの名前
from pathlib import Path
newname = Path(in_file).stem
print (newname)

soccer_2019_2014


In [26]:
# タイムスタンプ
import os
from datetime import datetime
t = os.path.getmtime(in_file)
# エポック秒をdatetimeに変換
dt = datetime.fromtimestamp(t)
datadate = dt.strftime('%Y%m%d%H%M')
print (datadate)

201910311138


In [27]:
# 出力
df_merged.to_csv("./U18_" + newname + "_" + datadate + ".csv", 
          index=False   # 行インデックスを削除
         )